In [12]:
import time
import os
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from PIL import Image
from torch.autograd import Variable

### Dataloader

In [2]:
class myDataset(Dataset):
    def __init__(self, csv_path, root_dir, status=1):
        self.root_dir = root_dir
        
        self.data_info = pd.read_csv(csv_path)
        #image path
        self.img_array = np.asarray(self.data_info.iloc[0:,1])
        #labels
        self.labels = np.asarray(self.data_info.iloc[0:,2], dtype=np.float32)
        #length of data
        self.data_len = len(self.data_info.index)
        #transformations on the batch
            #if status = 1 ===> transforms on train images
            #if status = 0 ===> transforms on val images
        if status == 1:
            self.transformations = transforms.Compose([transforms.Resize((256, 256)),
                                                       transforms.RandomRotation((90, 180)),
                                                       transforms.ToTensor()])
        elif status == 0:
            self.transformations = transforms.Compose([transforms.Resize((256, 256)),
                                                       transforms.ToTensor()])
        else:
            raise ValueError('Status should be 1 or 0')
    
    def __len__(self):
        return self.data_len
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.img_array[idx] + '.jpeg')
        image = Image.open(img_path,'r')
        img_as_tensor = self.transformations(image)
        
        return {'image':img_as_tensor,'label':self.labels[idx]}
        #return sample
    

### Train

In [4]:
def train_model(model, dataloader_train, dataloader_val, criterion, optimizer, scheduler, num_epochs):
    
    start_time = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print 'Epoch {}/{}'.format(epoch, num_epochs - 1)
        print '-'*10
        
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            num_correct = 0
            
            if phase == 'train':
                dataset = dataloader_train
            else:
                dataset = dataloader_val
                
            for idx, batch in enumerate(dataset):
                data = batch['image']
                data = Variable(data)
                labels = batch['label']
                labels = labels.long()
                labels = Variable(labels)
                if use_cuda:
                    data = data.cuda()
                    labels = labels.cuda()

                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    output = model(data)
                    loss = criterion(output, labels)
                    _, preds = torch.max(output.data, 1)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    
                running_loss += loss.item()*data.size(0)
                num_correct += torch.sum(preds == labels.data)
                del data, labels, output, preds
                torch.cuda.empty_cache()
                
            epoch_loss =  running_loss / dataset.__len__()
            epoch_acc = num_correct.double() / dataset.__len__()    
           
            print '{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc)
            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                
        print 
        print '='*70
        print
    time_elapsed = time.time() - start_time
    print 'Training complete in {:.0f}mins {:.0f}secs'.format(time_elapsed // 60, time_elapsed % 60)
    print 'Best accuracy {:.4f}'.format(best_acc * 100)
    
    model.load_state_dict(best_model_wts)
    return model, best_acc   

### Prepare data

In [6]:
#csv_path = '/media/eslam/7AE0CA83E0CA455B/Users/Eslam100/Documents/kaggledataset/10k_part/train/labels.csv'
root_dir = '/media/eslam/7AE0CA83E0CA455B/Users/Eslam100/Documents/kaggledataset/10k_part/'
trainset = myDataset(os.path.join(root_dir,'train/labels.csv'), os.path.join(root_dir,'train'))
dataloader_train  = DataLoader(dataset=trainset,
                         batch_size=10,
                         shuffle=False,
                         num_workers=4,
                         pin_memory=True)
val_set = myDataset(os.path.join(root_dir,'val/labels.csv'), os.path.join(root_dir,'val'))
dataloader_val = DataLoader(dataset=val_set,
                         batch_size=10,
                         shuffle=False,
                         num_workers=4,
                         pin_memory=True)

### Model

In [9]:
class ConvNet(nn.Module):
    
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.conv2 = nn.Conv2d(32, 32, 3)
        self.pool1 = nn.MaxPool2d(2, 2)

        self.conv3 = nn.Conv2d(32, 64, 3)
        self.conv4 = nn.Conv2d(64, 64, 3)
        self.pool2 = nn.MaxPool2d(2, 2)

        self.conv5 = nn.Conv2d(64, 128, 3)
        self.conv6 = nn.Conv2d(128, 128, 3)
        self.pool3 = nn.MaxPool2d(2, 2)

        self.conv7 = nn.Conv2d(128, 256, 3)
        self.conv8 = nn.Conv2d(256, 256, 3)
        self.pool4 = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(256*12*12,4096)
        self.fc2 = nn.Linear(4096, 5)
    
    def forward(self, input):
        x = self.pool1(self.conv2(F.relu(self.conv1(input))))
        x = self.pool2(self.conv4(F.relu(self.conv3(x))))
        x = self.pool3(self.conv6(F.relu(self.conv5(x))))
        x = self.pool4(self.conv8(F.relu(self.conv7(x))))
        #flatten the tensor for the FC
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    
    

### Main

In [13]:
use_cuda = torch.cuda.is_available()
model = ConvNet().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
criterion = nn.NLLLoss().cuda()
accuracy =0
mymodel, accuracy=train_model(model, dataloader_train, dataloader_val, criterion, optimizer, scheduler, num_epochs=25)

Epoch 0/24
----------


RuntimeError: cuda runtime error (2) : out of memory at /opt/conda/conda-bld/pytorch_1524577177097/work/aten/src/THC/generic/THCStorage.cu:58

In [8]:
dataloader_val.__len__()

200